# Tarea 3 : Mongo DB arXiv
| ROL         | Nombre          |
|-------------|-----------------|
| 202373037-8 | Matias Peñaloza |
| 202373020-3 | Hans González   |

In [29]:
from pymongo import MongoClient
from pymongo.cursor import Cursor

In [30]:
client = MongoClient("mongodb://mongo1:30001,mongo2:30002,mongo3:30003/?replicaSet=my-replica-set&readPreference=primary&appname=MongoDB%20Compass&ssl=false")
db = client["arxiv_db"]
collection = db["articles"]

In [31]:
import json
from IPython.display import display, HTML

def __print_article(article, fields):
    if "_id" in fields:
        article["_id"] = str(article["_id"])
    filtered = {field: article[field] for field in fields if field in article}
    json_str = json.dumps(filtered, indent=2, ensure_ascii=False)
    json_str = json_str.replace("<", "&lt;").replace(">", "&gt;")
    html = f"<pre>{json_str}</pre><hr>"
    display(HTML(html))

def print_articles(articles:Cursor, debug=False):
    articles = list(articles)
    print(f"Total de artículos encontrados: {len(articles)}")
    if not articles:
        display(HTML("<b>No se encontraron artículos.</b>"))
        return
    fields = [field for field in articles[0].keys() if field != "_id" or debug]
    for article in articles:
        __print_article(article, fields)

# Consultas
### Tipo de documento
```js
{
  "_id": ObjectId,
  "id": string,
  "submitter": string,
  "authors": string,
  "title": string,
  "comments": string,
  "journal-ref": string,
  "doi": string,
  "report-no": string,
  "categories": string,
  "license": string,
  "abstract": string,
  "versions": array,
  "update_date": string, // (YYYY-MM-DD)
  "authors_parsed": array,
  "pdf_source": string
}
```
#### a. Devolver los títulos y fechas de creación de artículos publicados en el año 2025. Mostrar solo esos campos y limitar a los primeros 20 resultados.

In [32]:
res = collection.find({
    "update_date": {
        "$gte": "2025-01-01",
        "$lt": "2026-01-01"
    }
}, {
    "title": 1,
    "update_date": 1
},
    limit=20
)
print_articles(res)

Total de artículos encontrados: 20


#### b. Devolver los títulos y los autores de artículos que pertenezcan a las categorías "cs.AI" o "stat.ML" y que tengan al menos tres autores. Mostrar solo esos campos y limitar a los primeros 10 resultados.

In [33]:
res = collection.find({
    "$expr": {
        "$gte": [3 , {"$size": "$authors_parsed"}]
    },
    "categories": {
        "$in": ["cs.AI", "stat.ML"]
    }
}, {
    "title": 1,
    "authors_parsed": 1,
},
    limit=10
)
print_articles(res)

Total de artículos encontrados: 10


#### c. Devolver los títulos, las categorías y los enlaces al PDF de artículos que pertenezcan a la categoría "hep-ph" y tengan un DOI asignado. Mostrar solo esos campos y limitar a 15 resultados.


In [34]:
res = collection.find({
    "categories": "hep-ph",
    "doi": { "$ne": None }
}, {
    "title": 1,
    "categories": 1,
    "pdf_source": 1,
},
    limit=15
)
print_articles(res)

Total de artículos encontrados: 15


#### d. Devolver los títulos, nombres de los autores y la referencia de publicación (journal-ref) de los artículos que tengan un DOI asignado. Mostrar solo esos campos y ordenar los resultados alfabéticamente por título. Limitar a los primeros 20 resultados.


In [35]:
res = collection.find({
    "doi": {"$ne": None },
}, {
    "title": 1,
    "authors_parsed": 1,
    "journal-ref": 1,
},
    limit=20,
    sort=[
        ("title", 1)
    ]
)
print_articles(res)

Total de artículos encontrados: 20


#### e. Devolver los títulos y la fecha de la primera versión (versions.created) de los artículos enviados entre los años 2010 y 2015. Mostrar solo esos campos y limitar a los primeros 15 resultados.


In [36]:
res = collection.find({
    "update_date": {
        "$gte": "2010-01-01",
        "$lt": "2015-01-01"
    },
}, {
    "title": 1,
    "created": {
        "$arrayElemAt": ["$versions.created", 0]
    }
},
    limit=15
)
print_articles(res)

Total de artículos encontrados: 15


#### f. Devolver los títulos, comentarios y reportes técnicos (report-no) de artículos que tengan comentarios definidos y no nulos. Mostrar solo esos campos, ordenando por fecha de actualización (update_date) en orden descendente. Limitar a 10 resultados.

In [37]:
res = collection.find({
    "comments": {
        "$exists": True,
        "$ne": None
    },
}, {
    "title": 1,
    "comments": 1,
    "report-no": 1,
},
    limit=10,
    sort=[
        ("update_date", -1)
    ]
)
print_articles(res)

Total de artículos encontrados: 10
